In [1]:
%load_ext autoreload
%autoreload 2

import os
import time
import glob
import gdown
import torch
import torch.nn.functional as F
import pandas as pd
import numpy as np
import csv
import cv2
import itertools
import matplotlib.pyplot as plt
import skimage.feature as feature
import xlwings as xw
import torchvision.transforms as transforms

import random

#libraries for yolo
from pytorchyolo.models import load_model
from pytorchyolo.utils.transforms import Resize, DEFAULT_TRANSFORMS
from pytorchyolo.utils.utils import non_max_suppression
from pytorchyolo.utils.loss import compute_loss

from matplotlib.ticker import (FormatStrFormatter, AutoMinorLocator, FuncFormatter, )

<frozen importlib._bootstrap>:219: RuntimeWarning: scipy._lib.messagestream.MessageStream size changed, may indicate binary incompatibility. Expected 56 from C header, got 64 from PyObject


In [17]:
from scripts.facedetectors import MediaPipe, YuNet, YoloFace

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [18]:
mp = MediaPipe()
yn = YuNet()
yf = YoloFace()

[BoxCoords(x1=126, y1=169, x2=299, y2=342)] [BoxCoords(x1=120, y1=129, x2=296, y2=349)] [BoxCoords(x1=126, y1=129, x2=288, y2=349)]


## YOLOFace with FGSM

In [ ]:
# Patterned after FGSM tutorial (https://pytorch.org/tutorials/beginner/fgsm_tutorial.html)
# Define what device we are using
print("CUDA Available: ", torch.cuda.is_available())
device, model = load_model('./weights/yolo_face_sthanhng.cfg', "./weights/yolo_face_sthanhng.weights")

# Set the model in evaluation mode. In this case this is for the Dropout layers
model.eval()

epsilons = [0, .05]
use_cuda=True

## Image Feature Extraction

In [19]:
from scripts.imageattributes import extract_gradients

In [ ]:
def detach_cpu(image):
    return image.detach().cpu()

# convert 1x3x416x416 to 416x416x3
def reshape_image(image):
    return np.transpose(np.squeeze(image), (1 ,2, 0))

# convert 1x3x416x416 tensor to 416x416x3 numpy image
def tensor_to_image(image):
    return np.transpose(image.detach().cpu().squeeze().numpy(), (1, 2, 0))

def save_tensor_as_image(image, path):
    save_img = cv2.cvtColor(np.moveaxis((image.detach().numpy() * 255).squeeze(), 0, -1).astype('uint8'), cv2.COLOR_RGB2BGR)
    cv2.imwrite(path, save_img)

In [3]:
import scripts.fgsm as fgsm

ModuleNotFoundError: No module named 'pytorchyolo'

In [ ]:
faces_df = ""

def load_mask(filename, face_num, target_bbox):
    filename = "restored_mask_" + os.path.splitext(filename)[0] + "_" + str(face_num) + "_image_final.png"
    mask = cv2.imread(os.path.join(os.getcwd(), RESTORED_MASK_PATH, filename), 0)
    
    face_row = faces_df.loc[faces_df['filename'] == filename]
    padded_dim = (int(face_row["x2_pad"] - face_row["x1_pad"]), int(face_row["y2_pad"] - face_row["y1_pad"]))
    target_dim = (int(target_bbox[2] - target_bbox[0]), int(target_bbox[3] - target_bbox[1]))
    
    #print(dict(zip(*np.unique(mask, return_counts = True)))[255], int(target_dim[0] * target_dim[1] * 0.1))
    #print(dict(zip(*np.unique(mask, return_counts = True)))[255] < int(target_dim[0] * target_dim[1] * 0.1))
    
    if dict(zip(*np.unique(mask, return_counts = True)))[255] < int(target_dim[0] * target_dim[1] * 0.1):
        return torch.ones((1, 3, target_dim[1], target_dim[0])), False
    
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (int(mask.shape[0] * 0.5), int(mask.shape[1] * 0.5)))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)
    mask = cv2.cvtColor(mask, cv2.COLOR_GRAY2RGB)
    mask = transforms.Compose([DEFAULT_TRANSFORMS])((mask, np.zeros((1, 5))))[0].unsqueeze(0)
    
    #print(os.path.join(os.getcwd(), RESTORED_MASK_PATH, filename))
    #print("mask size before:", mask.shape)
    
    
    #print("x1, y1, orig shape")
    #print(int(face_row["x1_pad"]), int(face_row["y1_pad"]), int(face_row["x2_pad"]), int(face_row["y2_pad"]))
    #print(original_shape)
    
    #print("target shape:", original_shape)
    #print("yoloshape:", int(face_row["x2"] - face_row["x1"]), int(face_row["y2"] - face_row["y1"]))
    
    current_dim = max(mask.shape)
    diff_x, diff_y = abs(padded_dim[0] - current_dim) / 2, abs(padded_dim[1] - current_dim) / 2
    #print("first diff:", diff_x, diff_y)
    
    if diff_y != 0:
        mask = mask[..., int(np.floor(diff_y)):-int(np.ceil(diff_y)), :]
    if diff_x != 0:
        mask = mask[..., int(np.floor(diff_x)):-int(np.ceil(diff_x))]
        
    #print(mask.shape == padded_dim, mask.shape, padded_dim, target_dim)
    
    padding = [
        int(abs(face_row["x1"] - face_row["x1_pad"])),
        int(abs(face_row["y1"] - face_row["y1_pad"])),
        int(abs(face_row["x2"] - face_row["x2_pad"])),
        int(abs(face_row["y2"] - face_row["y2_pad"]))
    ]
    
    #print("padding:", padding)
    
    new_dim = padded_dim[0] - padding[0] - padding[2], padded_dim[1] - padding[1] - padding[3]
    diff_x, diff_y = (target_dim[0] - new_dim[0]) / 2, (target_dim[1] - new_dim[1]) / 2
    #print("second diff:", diff_x, diff_y)
    
    padding[0] -= int(np.floor(diff_x))
    padding[1] -= int(np.floor(diff_y))
    padding[2] -= int(np.ceil(diff_x))
    padding[3] -= int(np.ceil(diff_y))
    
    #print("mask size after:", mask.shape)
    #print("unpadded bbox:", (face_row["x1"], face_row["y1"], face_row["x2"], face_row["y2"]))
    #print("adjusted padding:", padding)
    # mask = mask[..., padding[1]:-padding[3], padding[0]:-padding[2]]
    
    mask = F.pad(input=mask, pad=(-padding[0], -padding[2], -padding[1], -padding[3]), mode='constant', value=0)
    
    return mask, True

In [ ]:
def pipeline(model, device):
    
    torch.autograd.set_detect_anomaly(True)
    
    df = pd.DataFrame() # dataframe storing the dataset
    row = {} #the information/columns for a single row in the dataset is stored here
    
    # Loop over all examples in test set
    for path in glob.glob(os.path.join(INPUT_PATH, '*.jpg')):
        row['path'] = path
        file_basename = os.path.basename(path)
        print(file_basename)
        
        REF_SUBSET = REF_SET[REF_SET['source_file'] == file_basename]
        
        if len(REF_SUBSET.index) > 0:
            model.eval()

            model.gradient_mode = False
            for yolo_layer in model.yolo_layers:
                yolo_layer.gradient_mode = False

            # read and transform the image from the path
            data = cv2.imread(path)  # read the image
            row['source_w'], row['source_h'], _ = data.shape
            data = cv2.cvtColor(data, cv2.COLOR_BGR2RGB) #change to rgb
            data = transforms.Compose([DEFAULT_TRANSFORMS,Resize(416)])((data, np.zeros((1, 5))))[0].unsqueeze(0) # transform the image
            
            with torch.no_grad():
                # Forward pass the data through the model and call non max suppression
                nms, nms_output = non_max_suppression(model(data), 0.5, 0.5) #conf_thres and iou_thres = 0.5

            face_list = []
            print(nms_output)
            if type(nms_output[0]) is not int:
                face_list = nms_output[0]

            data = data.to(device)
            # Set requires_grad attribute of tensor. Important for Attack
            data.requires_grad = True

            model.gradient_mode = True
            for yolo_layer in model.yolo_layers:
                yolo_layer.gradient_mode = True

            output = model(data)

            # loop through each of the faces in the image
            for face_index, face_row in enumerate(face_list): #nms_output[0] because the model is designed to take in several images at a time from the dataloader but we are only loading the image one at a time
                if face_index in set(REF_SUBSET['face_index']):

                    row['face_index'] = face_index
                    print("Face", face_index)

                    row['obj_score'] = face_row[4].item()
                    row['class_score'] = face_row[5].item()
                    x, y, w, h = face_row[0], face_row[1], face_row[2], face_row[3]

#                     factor_x, factor_y, factor_w, factor_h = random.uniform(1, 2), random.uniform(1, 2), random.uniform(1, 2), random.uniform(1, 2)
                    normal_x, normal_y, normal_w, normal_h = x / 416, y / 416, w / 416, h / 416

#                     new_x = normal_x * factor_x if random.choice([True, False]) else normal_x / factor_x
#                     new_y = normal_y * factor_y if random.choice([True, False]) else normal_y / factor_y
#                     new_w = normal_w * factor_w if random.choice([True, False]) else normal_w / factor_w
#                     new_h = normal_h * factor_h if random.choice([True, False]) else normal_h / factor_h

#                     new_x, new_y, new_w, new_h = max(min(1, new_x), 0), max(min(1, new_y), 0), max(min(1, new_w), 0), max(min(1, new_h), 0)

                    target = torch.tensor([[0.0, 0, normal_x, normal_y, normal_w, normal_h]])
                    target = target.to(device)

                    loss, loss_components = compute_loss(output, target, model)

                    # cropped image with bounding box
                    # getting (x1, y1) upper left, (x2, y2) lower right
                    x1 = max(int(np.floor((x - w / 2).detach().cpu().numpy())), 0)
                    y1 = max(int(np.floor((y - h / 2).detach().cpu().numpy())), 0)
                    x2 = min(int(np.ceil((x + w / 2).detach().cpu().numpy())), 415)
                    y2 = min(int(np.ceil((y + h / 2).detach().cpu().numpy())), 415)

                    row['x1'], row['y1'], row['x2'], row['y2'] = x1, y1, x2, y2

                    cropped_image = detach_cpu(data)[:, :, y1:y2, x1:x2] #get the first dimension, the channels, and crop it
                    cropped_image = tensor_to_image(cropped_image) #reshape the image to (w/h, h/w, channel)

                    # Zero all existing gradients
                    model.zero_grad()
                    data.grad = None

                    # Calculate gradients of model in backward pass
                    loss.backward(retain_graph=True) #TODO: Amos - check if this is correct

                    # Collect datagrad
                    data_grad = data.grad.data
                    #print('Gradient')
                    #print(data_grad)
                    #print(data_grad.shape)
                    #plt.imshow(np.transpose(np.clip(data_grad.squeeze(0).numpy(), 0, 1), (1, 2, 0)))
                    #plt.show()

                    bbox = (x1, y1, x2, y2)
                    mask, used_mask = load_mask(os.path.basename(path), face_index, bbox)
                    row['used_mask'] = used_mask


                    #TODO: Jay - extract image attributes here
                    # extract the image attributes from  the 'cropped_image' variable
                    # save the attributes as row['<column name in the dataset>'] = <data> (see examples above for reference)

                    row = extract_image_attributes(row, path, face_index, cropped_image * tensor_to_image(mask[0]), "mask")
                    row = extract_image_attributes(row, path, face_index, cropped_image, "bbox")

                    whole_mask = np.zeros(data.shape)
                    whole_mask[..., y1:y2, x1:x2] = mask
#                     inverted_mask = np.zeros(data.shape)
#                     inverted_mask[..., y1:y2, x1:x2] = (1 - whole_mask[..., y1:y2, x1:x2]) if used_mask else whole_mask[..., y1:y2, x1:x2]

                    #print("bbox dim:", bbox)
                    #print(mask.shape, data[:, :, y1:y2, x1:x2].shape)
                    # TODO - Amos - determine the value of epsilon by calling fgsm_attack and changing the value of epsilon (see code below)
                    # the value of data(image) and data_grad remains constant diba

                    #print("Calculating min epsilon for YuNet...")
                    yn_min_e_face = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yn_det_fn, whole_mask, bbox)
                    #print("Calculating min epsilon for MediaPipe...")
#                     mp_min_e_face = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.mp_det_fn, whole_mask, bbox)
                    #print("Calculating min epsilon for YoloFace...")
#                     yf_min_e_face = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, whole_mask, bbox)
                    
#                     print("yn min face:", yn_min_e_face, "mp min face:", mp_min_e_face, "yf min face:", yf_min_e_face)
#                     row['e_face_yn'], row['e_face_mp'], row['e_face_yf'] = yn_min_e_face, mp_min_e_face, yf_min_e_face
                    
                    print("yn min face:", yn_min_e_face)
                    row['e_face_yn'] = yn_min_e_face

#                     if used_mask:
#                         yn_min_e_bg = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yn_det_fn, inverted_mask, bbox, background=True)
#                         mp_min_e_bg = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.mp_det_fn, inverted_mask, bbox, background=True)
#                         yf_min_e_bg = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, inverted_mask, bbox, background=True)
#                     else:
#                         yn_min_e_bg = yn_min_e_face
#                         mp_min_e_bg = mp_min_e_face
#                         yf_min_e_bg = yf_min_e_face

#                     print("yn min bg:", yn_min_e_bg, "mp min bg:", mp_min_e_bg, "yf min bg:", yf_min_e_bg, "used mask" if used_mask else "did not use mask")
#                     row['e_bg_yn'], row['e_bg_mp'], row['e_bg_yf'] = yn_min_e_bg, mp_min_e_bg, yf_min_e_bg

                    bbox_mask = np.zeros(data.shape)
                    bbox_mask[..., y1:y2, x1:x2] = 1

                    if used_mask:
                        yn_min_e_bbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yn_det_fn, bbox_mask, bbox)
#                         mp_min_e_bbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.mp_det_fn, bbox_mask, bbox)
#                         yf_min_e_bbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, bbox_mask, bbox)
                    else:
                        yn_min_e_bbox = yn_min_e_face
#                         mp_min_e_bbox = mp_min_e_face
#                         yf_min_e_bbox = yf_min_e_face

#                     print("yn min bbox:", yn_min_e_bbox, "mp min bbox:", mp_min_e_bbox, "yf min bbox:", yf_min_e_bbox)
#                     row['e_bbox_yn'], row['e_bbox_mp'], row['e_bbox_yf'] = yn_min_e_bbox, mp_min_e_bbox, yf_min_e_bbox
                    
                    print("yn min bbox:", yn_min_e_bbox)
                    row['e_bbox_yn'] = yn_min_e_bbox 

                    large_x1 = max(int(np.floor((x - w).detach().cpu().numpy())), 0)
                    large_y1 = max(int(np.floor((y - h).detach().cpu().numpy())), 0)
                    large_x2 = min(int(np.ceil((x + w).detach().cpu().numpy())), 415)
                    large_y2 = min(int(np.ceil((y + h).detach().cpu().numpy())), 415)

                    large_bbox_mask = np.zeros(data.shape)
                    large_bbox_mask[..., large_y1:large_y2, large_x1:large_x2] = 1

                    yn_min_e_lbbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yn_det_fn, large_bbox_mask, bbox)
#                     mp_min_e_lbbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.mp_det_fn, large_bbox_mask, bbox)
#                     yf_min_e_lbbox = minE.min_model_eps(data.clone().detach(), data_grad.clone().detach(), minE.yf_det_fn, large_bbox_mask, bbox)

#                     print("yn min lbbox:", yn_min_e_lbbox, "mp min lbbox:", mp_min_e_lbbox, "yf min lbbox:", yf_min_e_lbbox)
#                     row['e_lbbox_yn'], row['e_lbbox_mp'], row['e_lbbox_yf'] = yn_min_e_lbbox, mp_min_e_lbbox, yf_min_e_lbbox
                    
                    print("yn min lbbox:", yn_min_e_lbbox)
                    row['e_lbbox_yn'] = yn_min_e_lbbox

#                     cur_filename_base = os.path.join(PERTURBED_OUTS, os.path.splitext(file_basename)[0] + "_" + str(face_index))

#                     cur_filename =  cur_filename_base + "_yf_min_e_face.png"
#                     save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yf_min_e_face, data_grad.clone().detach(), whole_mask), cur_filename)
#                     cur_filename =  cur_filename_base + "_yf_min_e_bg.png"
#                     save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yf_min_e_bg, data_grad.clone().detach(), inverted_mask), cur_filename)
#                     cur_filename =  cur_filename_base + "_yf_min_e_bbox.png"
#                     save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yf_min_e_bbox, data_grad.clone().detach(), bbox_mask), cur_filename)
#                     cur_filename =  cur_filename_base + "_yf_min_e_lbbox.png"
#                     save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yf_min_e_lbbox, data_grad.clone().detach(), large_bbox_mask), cur_filename)


        #             cur_filename =  cur_filename_base + "yn_min_e_face.png"
        #             save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yn_min_e_face, data_grad.clone().detach(), mask, *bbox), cur_filename)
        #             cur_filename =  cur_filename_base + "mp_min_e_face.png"
        #             save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), mp_min_e_face, data_grad.clone().detach(), mask, *bbox), cur_filename)
        #             cur_filename =  cur_filename_base + "yf_min_e_face.png"
        #             save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yf_min_e_face, data_grad.clone().detach(), mask, *bbox), cur_filename)
        #             cur_filename =  cur_filename_base + "yn_min_e_bg.png"
        #             save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yn_min_e_bg, data_grad.clone().detach(), inverted_mask, *bbox), cur_filename)
        #             cur_filename =  cur_filename_base + "mp_min_e_bg.png"
        #             save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), mp_min_e_bg, data_grad.clone().detach(), inverted_mask, *bbox), cur_filename)
        #             cur_filename =  cur_filename_base + "yf_min_e_bg.png"
        #             save_tensor_as_image(minE.fgsm_attack(data.clone().detach(), yf_min_e_bg, data_grad.clone().detach(), inverted_mask, *bbox), cur_filename)

                    # Call FGSM Attack

                    #perturbed_data = minE.fgsm_attack(data.clone().detach(), yf_min_e, data_grad.clone().detach(), mask, *bbox)
                    #perturbed_data = fgsm_attack(data, max(yn_min_e, mp_min_e), data_grad) #data is the input image, epsilon
                    #print("can detect faces on unperturbed img?", minE.mp_det_fn(data.detach()))
                    #print(f"can detect faces on perturbed data with e={max(yn_min_e, mp_min_e) - 0.01}?", minE.mp_det_fn(fgsm_attack(data, max(yn_min_e, mp_min_e) - 0.01, data_grad).detach()))
                    #print(f"can detect faces on perturbed img? with e={max(yn_min_e, mp_min_e) - 0.01}", minE.mp_det_fn(perturbed_data.detach()))

                    df = df.append(row, ignore_index=True) #append the attributes of one face to the dataframe
            
    df.to_csv(os.path.join(CSV_PATH, FOLDER_NAME + 'yunet_features_dataset' + str(int(time.time())) + '.csv'), index=False)  #save to csv

In [ ]:
folders = []
fails = []
folders = ["img_celeba_94", "img_celeba_93"]
OUTPUT_FOLDER = os.path.join(os.getcwd(), "faceseg-outs")

REF_SET = pd.read_csv(os.path.join(os.getcwd(), 'reference_dataset.csv'), index_col=0)
REF_SET.reset_index()
REF_SET.head()

for FOLDER_NAME in folders:
    INPUT_PATH = os.path.join(os.getcwd(), 'images', FOLDER_NAME)
    FOLDER_PATH = os.path.join(OUTPUT_FOLDER, FOLDER_NAME)
    CSV_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_CSV')
    RESTORED_MASK_PATH = os.path.join(FOLDER_PATH, FOLDER_NAME + '_restored_mask')
    PERTURBED_OUTS = os.path.join(FOLDER_PATH, FOLDER_NAME + "_perturbed_outs")
    
    if not os.path.exists(PERTURBED_OUTS):
        os.mkdir(PERTURBED_OUTS)
    
    CSV_FILE = ""
    for file in os.listdir(CSV_PATH):
        if "dataset_pixels" in file and file.endswith(".csv"):
            CSV_FILE = os.path.join(os.getcwd(), CSV_PATH, file)
            
    faces_df = pd.read_csv(CSV_FILE)
    faces_df.loc[:, ["x1", "y1", "x2", "y2", "x1_pad", "y1_pad", "x2_pad", "y2_pad"]] = faces_df.loc[:, ["x1", "y1", "x2", "y2", "x1_pad", "y1_pad", "x2_pad", "y2_pad"]].clip(lower = 0)
    
    #print(CSV_FILE)
    #raise ex
    print("Working on", FOLDER_NAME, "folder")
    try:
        pipeline(model, device)
    except:
        fails += FOLDER_NAME
        print("An exception occurred")

##### ___